In [1]:
import numpy as np
import pandas as pd
# import pathlib
import shutil
import os
import zipfile

import matplotlib.pyplot as plt
# import seaborn as sns
import tensorflow as tf
# from sklearn.metrics import confusion_matrix,classification_report

In [2]:
train_zip = '../input/dogs-vs-cats/train.zip'
tzip_ref = zipfile.ZipFile(train_zip,'r')
tzip_ref.extractall('./')
tzip_ref.close()

test_zip = '../input/dogs-vs-cats/test1.zip'
tzip_ref = zipfile.ZipFile(test_zip,'r')
tzip_ref.extractall('./')
tzip_ref.close()

os.mkdir('./separated')

source = "./separated"
for f in os.listdir('./train'):
    folderName = f[0:3]
    
    if not os.path.exists(f"{source}/{folderName}"):
        # f"{source}/{file}"
        os.mkdir(f"{source}/{folderName}")
        shutil.copy(os.path.join('./train',f),f"{source}/{folderName}")
    else:
        shutil.copy(os.path.join('./train',f),f"{source}/{folderName}")    

# Load Image Data


In [3]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                                                 horizontal_flip=True,
                                                                 width_shift_range=0.2,
                                                                 height_shift_range=0.2,
                                                                 shear_range=0.2,
                                                                 zoom_range=0.2,
                                                                 validation_split=0.2)
test_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)


In [4]:
train_images = train_generator.flow_from_directory('./separated',
                                   target_size=(150,150),
                                   batch_size=64,
                                   class_mode = 'binary',
                                   color_mode='rgb',
                                   subset='training'
                                   )

val_images = train_generator.flow_from_directory('./separated',
                                   target_size=(150,150),
                                   batch_size=32,
                                   class_mode = 'binary',
                                   color_mode='rgb',
                                   subset='validation'
                                   )

# test_images = test_generator.flow_from_directory('./test1',
#                                                 target_size=(150,150),
#                                                 batch_size=32,
#                                                 class_mode = 'binary',
#                                                 color_mode='rgb',
#                                                 )

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [5]:
inputs = tf.keras.Input(shape=(150,150,3))

x = tf.keras.layers.Conv2D(filters=16, kernel_size=(3,3),activation="relu")(inputs)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3),activation="relu")(x)
x = tf.keras.layers.MaxPool2D()(x)

x=tf.keras.layers.Flatten()(x)

x = tf.keras.layers.Dense(128,activation='relu')(x)
x = tf.keras.layers.Dense(128,activation='relu')(x)
outputs = tf.keras.layers.Dense(1,activation="sigmoid")(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])

history = model.fit(train_images,
                   validation_data = val_images,
                   epochs=20,
                   callbacks=[
                       tf.keras.callbacks.EarlyStopping(
                           monitor='val_loss',
                           patience=5,
                           restore_best_weights=True
                       ),
                       tf.keras.callbacks.ReduceLROnPlateau(
                           monitor='val_loss',
                           patience=2
                       )
                   ]
                   )

Epoch 1/20
313/313 [==============================] - 176s 542ms/step - loss: 0.6822 - accuracy: 0.5832 - val_loss: 0.6032 - val_accuracy: 0.6708
Epoch 2/20
313/313 [==============================] - 169s 541ms/step - loss: 0.5860 - accuracy: 0.6863 - val_loss: 0.5516 - val_accuracy: 0.7202
Epoch 3/20
313/313 [==============================] - 168s 538ms/step - loss: 0.5483 - accuracy: 0.7207 - val_loss: 0.5181 - val_accuracy: 0.7484
Epoch 4/20
313/313 [==============================] - 168s 537ms/step - loss: 0.5381 - accuracy: 0.7256 - val_loss: 0.5328 - val_accuracy: 0.7262
Epoch 5/20
313/313 [==============================] - 168s 536ms/step - loss: 0.5160 - accuracy: 0.7442 - val_loss: 0.5025 - val_accuracy: 0.7526
Epoch 6/20
313/313 [==============================] - 167s 535ms/step - loss: 0.5059 - accuracy: 0.7484 - val_loss: 0.4913 - val_accuracy: 0.7654
Epoch 7/20
313/313 [==============================] - 167s 535ms/step - loss: 0.4866 - accuracy: 0.7634 - val_loss: 0.4754 -

In [6]:
model_json = model.to_json()
with open("./model.json","w") as json_file:
  json_file.write(model_json)

# Step 13: Save the weights in a seperate file
model.save_weights("./model.h5")

print("Classifier trained Successfully!")

Classifier trained Successfully!
